In [94]:
import numpy as np
import matplotlib.pyplot as plt
import sys
from sklearn.datasets import load_iris

In [95]:
from micrograd.engine import Value
from micrograd.nn import MLP

In [96]:
sys.setrecursionlimit(10000)

In [97]:
iris = load_iris()

X = np.array(iris.data)
y = np.array(iris.target)

r_index= list(np.random.permutation(len(y)))

X_train, y_train = X[r_index[:100]], y[r_index[:100]]
X_test, y_test = X[r_index[100:]], y[r_index[100:]]

print("Dataset dimension:")

print("Training set = ", len(X_train), len(y_train))
print("Test set = ", len(X_test), len(y_test))

print("Dimension of input = ", len(X_train[0]))

Dataset dimension:
Training set =  100 100
Test set =  50 50
Dimension of input =  4


In [98]:
print("Feature names:", iris.feature_names)
print("Target names:", iris.target_names)

print("First 5 samples:")
for i in range(5):
    print(f"Sample {i+1}: {X[i]} (Class: {y[i]}, Species: {iris.target_names[y[i]]})")

Feature names: ['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']
Target names: ['setosa' 'versicolor' 'virginica']
First 5 samples:
Sample 1: [5.1 3.5 1.4 0.2] (Class: 0, Species: setosa)
Sample 2: [4.9 3.  1.4 0.2] (Class: 0, Species: setosa)
Sample 3: [4.7 3.2 1.3 0.2] (Class: 0, Species: setosa)
Sample 4: [4.6 3.1 1.5 0.2] (Class: 0, Species: setosa)
Sample 5: [5.  3.6 1.4 0.2] (Class: 0, Species: setosa)


In [99]:
model = MLP(4, [3])
print('number of nodes', len(model.parameters()))
print(model)

number of nodes 15
MLP of [Layer of [Softmax Neuron (4), Softmax Neuron (4), Softmax Neuron (4)]]


In [100]:
# loss function
def loss():
    inputs = [list(map(Value, xrow)) for xrow in X_train]
    
    # forward the model to get scores
    scores = list(map(model, inputs)) 
    
    # Cross-entropy Loss
    losses = [(- score[yi].log()) for yi, score in zip(y_train, scores)]
    total_loss = sum(losses) / len(losses)

    # Accuracy
    accuracy = []
    for yi, score in zip(y_train, scores):
        argmax = -1
        valmax = 0
        for idx, scorei in enumerate(score):
            if scorei.data > valmax:
                valmax = scorei.data
                argmax = idx
        accuracy.append((argmax == yi))

    acc = sum(accuracy) / len(accuracy)
        
    return total_loss, acc

total_loss, acc = loss()
print(f"step {0} loss {total_loss.data}, accuracy {acc*100}%")

step 0 loss 3.0833045531652625, accuracy 35.0%


In [101]:
# backward
for k in range(100):
    model.zero_grad()
    total_loss.backward()
        
    # update (sgd)
    learning_rate = 1.0 - 0.9   *k/100
    for p in model.parameters():
        p.data -= learning_rate * p.grad

    total_loss, acc = loss()
        
    print(f"step {k + 1} loss {total_loss.data}, accuracy {acc*100}%")

step 1 loss 12.289526460019374, accuracy 36.0%
step 2 loss 23.92474759790549, accuracy 32.0%
step 3 loss 18.7361544165299, accuracy 32.0%
step 4 loss 11.705742805898574, accuracy 32.0%
step 5 loss 14.929623785721216, accuracy 36.0%
step 6 loss 14.249479248016986, accuracy 32.0%
step 7 loss 19.68421611758206, accuracy 32.0%
step 8 loss 3.003324116950807, accuracy 38.0%
step 9 loss 6.357328915001207, accuracy 32.0%
step 10 loss 13.813564492022067, accuracy 34.0%
step 11 loss 7.4900449795087605, accuracy 68.0%
step 12 loss 0.17714346068017225, accuracy 93.0%
step 13 loss 2.3312177058424575, accuracy 68.0%
step 14 loss 10.5621864747883, accuracy 68.0%
step 15 loss 0.1474070023467112, accuracy 95.0%
step 16 loss 1.1458250761942816, accuracy 68.0%
step 17 loss 10.321122883351766, accuracy 38.0%
step 18 loss 6.964933999055595, accuracy 68.0%
step 19 loss 3.032574672196091, accuracy 65.0%
step 20 loss 8.103835072730948, accuracy 68.0%
step 21 loss 1.435436262495156, accuracy 68.0%
step 22 loss

In [102]:
def test(batch_size=None):
    inputs = [list(map(Value, xrow)) for xrow in X_test]
    
    # forward the model to get scores
    scores = list(map(model, inputs))
    
    # Cross-entropy Loss
    losses = [(- score[yi].log()) for yi, score in zip(y_test, scores)]
    total_loss = sum(losses) / len(losses)

    # Accuracy
    predict = []
    accuracy = []
    for yi, score in zip(y_test, scores):
        argmax = -1
        valmax = 0
        for idx, scorei in enumerate(score):
            if scorei.data > valmax:
                valmax = scorei.data
                argmax = idx
        predict.append((argmax, yi))
        accuracy.append((argmax == yi))

    acc = sum(accuracy) / len(accuracy)
        
    return total_loss, acc, predict

In [104]:
# evaluate
total_loss, acc, pred = test()
print(f"Loss = {total_loss.data}, accuracy = {acc*100}%")

Loss = 0.08012238980901966, accuracy = 98.0%
